In [1]:
# Этап I. Обучение модели [ правильнее сказать - пополнение базы данных ]

#Получаем оригинальные данные из трейна

import os
import cv2
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from PIL import Image
from tqdm import tqdm

dir_path = '/home/user1/data/train Росатом'

# Путь к папкам
images_dir = os.path.join(dir_path, 'train/imgs')  # Папка с изображениями
labels_dir = os.path.join(dir_path, 'train/labels')  # Папка с разметками
labels_text_dir = os.path.join(dir_path, "train/labels_with_text")

# Список для хранения данных таблицы
data = []

# Обход папки с изображениями
for image_file in os.listdir(images_dir):
    if image_file.endswith(('.png', '.jpg', '.jpeg', '.JPG')):
        image_path = os.path.join(images_dir, image_file)
        label_path = os.path.join(labels_dir, os.path.splitext(image_file)[0] + '.txt')
        label_text_path = os.path.join(labels_text_dir, os.path.splitext(image_file)[0] + '.bbox')

        # Проверка наличия файла разметки
        if os.path.exists(label_path):


            # Чтение текстовых меток
            if os.path.exists(label_text_path):
                with open(label_text_path, 'r') as f:
                    label_text = ''.join(f.readlines())
                    label_text = label_text.replace('\n', '')
            else:
                label_text = np.NaN


            # Чтение текстовых меток
            if os.path.exists(label_path):
                with open(label_path, 'r') as f:
                    label = ''.join(f.readlines())
            else:
                label = np.NaN

            # Добавление данных в таблицу
            data.append([image_file, label, label_text])

# Создание DataFrame с изображениями и текстовыми метками
df = pd.DataFrame(data, columns=['image_file', 'label', 'text'])
df['text'] = df['text'].apply(lambda x: x.replace('"',''))
df['article'] = df['text'].apply(lambda x: x.split(' ')[0]) #Разделяем артикул и номер
df['image_path'] = images_dir+ '/' + df['image_file']
df.head()


,image_file,label,text,article,image_path
0,301.JPG,0 0.495625 0.472500 0.556250 0.401667\n,АМ116.06.00.901-03,АМ116.06.00.901-03,/home/user1/data/train Росатом/train/imgs/301.JPG
1,299.jpg,0 0.477766 0.447012 0.378501 0.285247\n,АМ116.06.00.901-03 1,АМ116.06.00.901-03,/home/user1/data/train Росатом/train/imgs/299.jpg
2,393.jpg,0 0.464706 0.504657 0.326797 0.446569\n,АМ116.06.00.962 2,АМ116.06.00.962,/home/user1/data/train Росатом/train/imgs/393.jpg
3,453.jpg,0 0.489542 0.528676 0.762092 0.348529\n,АМ116.06.01.301 31,АМ116.06.01.301,/home/user1/data/train Росатом/train/imgs/453.jpg
4,60.JPG,0 0.476562 0.321573 0.703125 0.489919\n,1391-30-0109 241,1391-30-0109,/home/user1/data/train Росатом/train/imgs/60.JPG


In [3]:
# Получаем оригинальные данные из экселя, они нужны будут для формирования списка артикулов.

def apply_to_number(x):
    if x is not None:
        return str(x).split('.')[0]
    else:
        return x

data = pd.read_excel('./../details.xlsx')
data['ПорядковыйНомер'] = data['ПорядковыйНомер'].apply(apply_to_number)
parts_df = data[['ДетальАртикул', 'ПорядковыйНомер']]
parts_df.columns = ['article', 'number']
parts_df.head()
parts_df['text'] = parts_df['article'].apply(lambda x: x.replace('"','')) + ' ' + parts_df['number']
parts_df.head()


/tmp/ipykernel_309139/443440091.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  parts_df['text'] = parts_df['article'].apply(lambda x: x.replace('"','')) + ' ' + parts_df['number']


,article,number,text
0,"""1391-30-0108 ТС1.1""",75,1391-30-0108 ТС1.1 75
1,"""1391-30-0114 ТС1.1""",43,1391-30-0114 ТС1.1 43
2,"""1391-30-0115 ТС1.1""",55,1391-30-0115 ТС1.1 55
3,"""1391-30-1145 ТС1.1""",0,1391-30-1145 ТС1.1 0
4,"""1391-30-1146 ТС1.1""",223,1391-30-1146 ТС1.1 223


In [22]:
# В Данном ячейке описана логика работы OCR.
# Изучив множество библиотек, обучив свои модели и поэксперементировав наша команда пришла к выводу
# что лучшим решением по работе с такими данными будет PaddleOCR. 
#
# Кроме этого он работает достаточно быстро и хорошо масштабируется, т.к. он не обучался только на тех данных
# которые были представлены организаторами.

from paddleocr import PaddleOCR
import logging
import time
logging.getLogger("ppocr").setLevel(logging.ERROR)

# Инициализируем OCR       
ocr = PaddleOCR(
    use_angle_cls=False, 
    lang='en',
    ocr_version='PP-OCRv4',
    use_space_char=True,
    use_gpu=True,
    enable_mkldnn=True,  
    use_tensorrt=True, 
    enable_fp16=True,
)


def multi_angle_ocr(image, angles=[0, 45, 90, 135, 180, 225, 270, 315]):
    """Функция для распознавания текста на изображении с учетом различных углов"""
    start_total = time.time()
    
    best_text = ""
    max_confidence = 0
    best_rotated = None
    best_box = None
    best_angle = 0

    if image is None:
        return best_text, best_rotated, best_box

    height, width = image.shape[:2]
    max_dim = max(height, width)
    if max_dim > 640:
        scale = 640.0 / max_dim
        new_width = int(width * scale)
        new_height = int(height * scale)
        image = cv2.resize(image, (new_width, new_height))
    
    height, width = image.shape[:2]
    center = (width // 2, height // 2)

    for angle in angles:
        
        rotation_matrix = cv2.getRotationMatrix2D(center, angle, 1.0)
        rotated = cv2.warpAffine(image, rotation_matrix, (int(width), int(height)))
        result = ocr.ocr(rotated, cls=False)
        

        if result[0]:
            confidence = sum(line[1][1] for line in result[0])
            if confidence > max_confidence:
                max_confidence = confidence
                best_text = " ".join([line[1][0] for line in result[0]])
                best_rotated = rotated
                best_angle = angle

                boxes = np.array([line[0] for line in result[0]])
                min_x = np.min(boxes[:, :, 0])
                min_y = np.min(boxes[:, :, 1])
                max_x = np.max(boxes[:, :, 0])
                max_y = np.max(boxes[:, :, 1])
                
                width_box = max_x - min_x
                height_box = max_y - min_y
                center_x = min_x + width_box/2
                center_y = min_y + height_box/2
                
                
                points = np.array([[center_x - width_box/2, center_y - height_box/2],
                                 [center_x + width_box/2, center_y - height_box/2],
                                 [center_x + width_box/2, center_y + height_box/2],
                                 [center_x - width_box/2, center_y + height_box/2]])
                
                inv_rotation_matrix = cv2.getRotationMatrix2D(center, -best_angle, 1.0)
                
                ones = np.ones(shape=(len(points), 1))
                points_ones = np.hstack([points, ones])
                transformed_points = inv_rotation_matrix.dot(points_ones.T).T
                
                min_x = np.min(transformed_points[:, 0])
                min_y = np.min(transformed_points[:, 1])
                max_x = np.max(transformed_points[:, 0])
                max_y = np.max(transformed_points[:, 1])
                
                width_box = max_x - min_x
                height_box = max_y - min_y
                center_x = min_x + width_box/2
                center_y = min_y + height_box/2
                
                rel_center_x = center_x / width
                rel_center_y = center_y / height
                rel_width = width_box / width
                rel_height = height_box / height
                
                best_box = [rel_center_x, rel_center_y, rel_width, rel_height]
        
    return best_text, best_rotated, best_box

image = np.array(Image.open('./../test_data/459.jpg'))
best_text, rotated_image, box = multi_angle_ocr(image)

# В данной ячейке мы получаем векторные представления изображений и текста
# для последующего их использования в модели сравнения.


In [7]:
# Для сравнения текста и изображений мы используем модель CLIP.

from transformers import CLIPProcessor, CLIPModel
import torch
from PIL import Image

model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)
model.eval()

# Функция для получения векторных представлений изображений и текста
def get_clip_embeddings(image=None, text=None):
    result = {}
    with torch.no_grad():
        if image is not None:
            if isinstance(image, np.ndarray):
                image = Image.fromarray(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
            if isinstance(image, str):
                image = Image.open(image)
                
            inputs = processor(images=image, return_tensors="pt").to(device)
            image_features = model.get_image_features(**inputs)
            result['image_embedding'] = image_features.cpu().numpy()
        
        if text is not None and isinstance(text, str):
            inputs = processor(text=text, return_tensors="pt", padding=True).to(device)
            text_features = model.get_text_features(**inputs)
            result['text_embedding'] = text_features.cpu().numpy()
    
    return result



In [8]:
# Получаем векторные представления для каждого изображения в тренировочном датасете
# Также получаем векторные представления для каждого текста в тренировочном датасете? для того чтобы проводить множественное сравнение
df['embeddings_image'] = df['image_path'].apply(lambda x: get_clip_embeddings(image=x)['image_embedding'])
df['embeddings_text'] = df['text'].apply(lambda x: get_clip_embeddings(text=x)['text_embedding'])

In [9]:
# Применяем OCR к каждому изображению в тренировочном датасете
# Получаем текст для каждого изображения
df['ocr_text'] = ''
for i,d in tqdm(df.iterrows()):
    image_path = d['image_path']
    image = cv2.imread(image_path)
    best_text, rotated_image, boxes = multi_angle_ocr(image)
    df.loc[i, 'ocr_text'] = best_text

# Также получаем эмбеддинги для текста occr, чтобы у нас была возможность сравнивать релевантные текстовые представления
df['ocr_text_embeddings'] = df['ocr_text'].apply(lambda x: get_clip_embeddings(text=x[:77])['text_embedding'])

252it [07:28,  1.78s/it]


In [10]:
# Группируем детали по артикулу, считаем средние вектора. Они и будут являться основопологающими при выборе изображения.
grouped = df.groupby('article').agg({
    'image_file': lambda x: list(x),
    'text': 'count', # Сохраняем список файлов
    # 'ocr_text': lambda x: np.mean(x,axis=0),
    'ocr_text_embeddings': lambda x: np.mean(x, axis=0),
    'embeddings_image': lambda x: np.mean(x, axis=0),
    'embeddings_text': lambda x: np.mean(x, axis=0)
}).rename(columns={'text': 'count'})

# Сохраняем результат для каждого класса. Таким образом можно сказать что grouped - это наша база данных.
grouped.to_pickle('grouped.pkl')

# На этом этап "обучения" модели можно считать завершенным.


# Отличительной особенностью нашего решения является то, что нет необходимости множество раз дообучать модель.
# Для точного предскаазния артикула в базу данных необходимо всего лишь одно изображения.
# Если изображение не добавлять, то поиск будет осуществляться через OCR по тексту.

# Кроме этого можно предсказывать изображения даже не по артикулу, а просто по фотографии, как и производить его поиск.

# Если развить данную систему, добавив frontend-панель управлениия, то появится возможность динамически пополнять базу данных,
# следить за ее состоянием, производить поиск как по артикулу, так и по самому предмету, либо же по фотографии.

In [11]:
# Этап II. Инференс

import os
import cv2
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from PIL import Image
from tqdm import tqdm

# Загружаем базу данных
grouped = pd.read_pickle('grouped.pkl')

grouped.head()

,image_file,count,ocr_text_embeddings,embeddings_image,embeddings_text
article,,,,,
1391-30-0109,"[60.JPG, 81.JPG, 80.JPG, 59.JPG, 77.jpg, 70.JP...",15,"[[0.017365314, 0.11561554, 0.07099361, 0.20430...","[[-0.24761134, -0.0367497, 0.051416826, 0.1916...","[[-0.025438841, 0.060193364, 0.06883027, 0.150..."
1391-30-1139,"[410.JPG, 362.JPG]",2,"[[0.08157718, 0.22039655, 0.011820257, 0.33380...","[[-0.1021947, 0.12941961, 0.0753935, -0.105946...","[[0.07487033, -0.05096645, 0.049642235, 0.1283..."
1391-30-1140,"[373.JPG, 346.JPG, 411.JPG]",3,"[[0.055068266, 0.09756765, 0.053187296, 0.2884...","[[-0.08799681, 0.21026927, 0.16012101, -0.1164...","[[0.06178923, -0.011043355, -0.021948034, 0.17..."
1391-30-1151,[92.jpg],1,"[[0.100430995, 0.24313453, 0.12627964, 0.12950...","[[-0.15982224, 0.053742856, 0.14010802, -0.014...","[[0.093301624, -0.049591593, 0.032554477, 0.12..."
1753-30-0124,"[413.JPG, 384.JPG]",2,"[[-0.047549512, 0.18820047, 0.06791867, 0.1052...","[[-0.0197642, 0.18334894, 0.20647895, -0.00318...","[[-0.04941526, 0.23083419, 0.07926907, 0.16700..."


In [23]:
# Код для предсказания артикула по изображению

i = 16

# Функция для вычисления косинусного сходства
def calculate_cosine_similarity(embedding1, embedding2):
    # Ensure embeddings are 1D arrays
    embedding1 = embedding1.flatten()
    embedding2 = embedding2.flatten()
    
    dot_product = np.dot(embedding1, embedding2)
    
    # Calculate norms
    norm1 = np.linalg.norm(embedding1)
    norm2 = np.linalg.norm(embedding2)
    
    # Calculate cosine similarity
    similarity = dot_product / (norm1 * norm2)
    
    return similarity

# Функция для предсказания артикула по изображению  
def get_image_article_text(image_path):
    image = cv2.imread(image_path)
    
    # Применяем OCR, распознаем текст и bbox текста
    best_text, rotated_image, box = multi_angle_ocr(image)

    # 
    if rotated_image is not None:
        image = rotated_image

    # Получаем эмбеддинги изображения и найденного на картинке текста
    image_embedding = get_clip_embeddings(image=image)['image_embedding']
    text_embedding = get_clip_embeddings(text=best_text[:77])['text_embedding']

    # Считаем схожесть входного изображения с изображениями (артикулами) в базе
    image_distances = grouped['embeddings_image'].apply(lambda x: calculate_cosine_similarity(
        image_embedding,
        x 
    ))
    # Считаем схожесть текста распознанного OCR с таким же распознанным OCR текстом для каждого артикула в базе
    text_ocr_distances = grouped['ocr_text_embeddings'].apply(lambda x: calculate_cosine_similarity(
        text_embedding,
        x 
    ))

    # Считаем схожесть текста распознанного OCR с оригинальным текстом артикула для каждого артикула в базе
    text_article_distances = grouped['embeddings_text'].apply(lambda x: calculate_cosine_similarity(
        text_embedding,
        x 
    ))

    # Все эти сравнения помогут нам с высокой точносью найти к какому артикулу пренадлежит данный текст. 
    # Кроме этого, сравнивая текстовые значения можно понять корректность нахождения номера изделия.

    groups = pd.DataFrame(grouped.index, columns=['article'])
    groups['image_distance'] = image_distances.values
    groups['text_ocr_distance'] = text_ocr_distances.values
    groups['text_article_distance'] = text_article_distances.values
    # Считаем "скор" для каждого артикула. Больше всего основываемся на тексте
    groups['score'] = groups['image_distance']*0.5+groups['text_ocr_distance']*0.3+groups['text_article_distance']*0.2
    groups.sort_values(by='score', ascending=False, inplace=True)
    # Берем артикул с наибольшим скором.
    result = {'article': groups.iloc[0].article, 'text': best_text, 'image_distance': groups.iloc[0].image_distance, 'text_ocr_distance': groups.iloc[0].text_ocr_distance, 'text_article_distance': groups.iloc[0].text_article_distance, 'score': groups.iloc[0].score, 'bbox': box if box is not None else [0.5,0.5,0.5,0.5]}
    return result

result = get_image_article_text(image_path)

print(result)

{'article': '1391-30-0109', 'text': '900832 181026', 'image_distance': 0.9257674, 'text_ocr_distance': 0.96897537, 'text_article_distance': 0.9646914, 'score': 0.9465146, 'bbox': [0.41640625, 0.6729166666666667, 0.4328125, 0.29583333333333345]}


In [47]:
# Считаем метрики на датасете
# Учитывая что все значения мбеддингов усредненные - можно считать что \
# На данном датасете можно валидироваться.

from shapely.geometry import box
from shapely.ops import unary_union
from Levenshtein import ratio
import pandas as pd
import numpy as np

# Функция для преобразования строки label в список bbox
def parse_label(label):
    try:
        boxes = []
        for item in label.split('\n'):
            if item:
                _, x_center, y_center, width, height = map(float, item.split())
                boxes.append([
                    x_center - width / 2, y_center - height / 2,
                    x_center + width / 2, y_center + height / 2
                ])
        return boxes
    except:
        raise Exception("Неправильный формат строки 'label'. Ожидается формат: '0 x_center y_center width height\\n...'")

# Функция для вычисления совокупного IoU для нескольких bbox
def calculate_total_iou(predicted_boxes, true_boxes):
    print(predicted_boxes, true_boxes)
    predicted_polygons = [box(*pred_box) for pred_box in predicted_boxes]
    true_polygons = [box(*true_box) for true_box in true_boxes]
    predicted_union = unary_union(predicted_polygons)
    true_union = unary_union(true_polygons)
    intersection_area = predicted_union.intersection(true_union).area
    union_area = predicted_union.union(true_union).area
    total_iou = intersection_area / union_area if union_area != 0 else 0
    return total_iou

# Основная функция для расчета метрик
def calc_metrics(ground_truth, submission):
    ious = []
    character_accuracies = []
    complete_matches = []

    ground_truth['text_is_notna'] = ground_truth['label_text'].notna()

    for _, row in ground_truth.iterrows():
        image_file = row['image_file']
        sub_row = submission[submission['image_file'] == image_file]

        if not sub_row.empty:
            gt_boxes = parse_label(row['label'])
            pred_boxes = parse_label(sub_row.iloc[0]['label'])
            iou = calculate_total_iou(pred_boxes, gt_boxes)
            ious.append(iou)

            if row['text_is_notna']:
                char_acc = ratio(row['label_text'], sub_row.iloc[0]['label_text'])
                character_accuracies.append(char_acc)
                complete_matches.append(row['label_text'] == sub_row.iloc[0]['label_text'])

    # Заполняем отсутствующие значения
    ious.extend([0] * (len(ground_truth) - len(ious)))
    character_accuracies.extend([0] * (ground_truth['text_is_notna'].sum() - len(character_accuracies)))
    complete_matches.extend([0] * (ground_truth['text_is_notna'].sum() - len(complete_matches)))

    mean_iou = np.mean(ious)
    mean_character_accuracy = np.mean(character_accuracies)
    accuracy = np.mean(complete_matches)

    return {
        "Средний IoU рамок": mean_iou,
        "Средняя посимвольная точность текста": mean_character_accuracy,
        "Точность абсолютно верного распознавания текста": accuracy
    }

# Расчет финального балла
def calc_score(metrics):
    score = (metrics["Средний IoU рамок"] * 0.05 +
             metrics["Средняя посимвольная точность текста"] * 0.65 +
             metrics["Точность абсолютно верного распознавания текста"] * 0.3)
    return score


In [43]:
images_dir = '/home/user1/data/train Росатом/train/imgs'
labels_dir = '/home/user1/data/train Росатом/train/labels'
labels_text_dir = '/home/user1/data/train Росатом/train/labels_with_text'

data = []
for image_file in os.listdir(images_dir):
    if image_file.endswith(('.png', '.jpg', '.jpeg', '.JPG')):
        image_path = os.path.join(images_dir, image_file)
        label_path = os.path.join(labels_dir, os.path.splitext(image_file)[0] + '.txt')
        label_text_path = os.path.join(labels_text_dir, os.path.splitext(image_file)[0] + '.bbox')

        # Проверка наличия файла разметки
        if os.path.exists(label_path):


            # Чтение текстовых меток
            if os.path.exists(label_text_path):
                with open(label_text_path, 'r') as f:
                    label_text = ''.join(f.readlines())
                    label_text = label_text.replace('\n', '')
            else:
                label_text = np.NaN


            # Чтение текстовых меток
            if os.path.exists(label_path):
                with open(label_path, 'r') as f:
                    label = ''.join(f.readlines())
            else:
                label = np.NaN

            # Добавление данных в таблицу
            data.append([image_file, label, label_text])

# Создание DataFrame с изображениями и текстовыми метками
gt = pd.DataFrame(data, columns=['image_file', 'label', 'label_text'])
gt

,image_file,label,label_text
0,301.JPG,0 0.495625 0.472500 0.556250 0.401667\n,"""АМ116.06.00.901-03"""
1,299.jpg,0 0.477766 0.447012 0.378501 0.285247\n,"""АМ116.06.00.901-03 1"""
2,393.jpg,0 0.464706 0.504657 0.326797 0.446569\n,"""АМ116.06.00.962 2"""
3,453.jpg,0 0.489542 0.528676 0.762092 0.348529\n,"""АМ116.06.01.301 31"""
4,60.JPG,0 0.476562 0.321573 0.703125 0.489919\n,"""1391-30-0109 241"""
...,...,...,...
247,42.JPG,0 0.456771 0.376563 0.461458 0.103125\n,"""195-30-1285 2668"""
248,443.JPG,0 0.506250 0.518623 0.751250 0.712190\n,"""АМ116.06.00.902 2"""
249,417.JPG,0 0.459167 0.519062 0.640000 0.236875\n,"""АМ116.05.02.200 1"""
250,430.JPG,0 0.480208 0.497266 0.631250 0.202344\n,"""АМ116.05.02.200 5"""


In [42]:
sub = []
for image_file in tqdm(os.listdir(images_dir)):
    if image_file.endswith(('.png', '.jpg', '.jpeg', '.JPG')):
        image_path = os.path.join(images_dir, image_file)
        result = get_image_article_text(image_path)
        label = '0 ' + ' '.join(map(str, result['bbox']))+'\n'
        if result['image_distance']>0.88:
            text = '"'+result["text"]+'"' if result['text_article_distance']>0.98 else '"'+result["article"]+' "'
        else:
            text = result["text"]
        image_sub = {'image_file': image_file, 'label': label, 'label_text': text}
        sub.append(image_sub)

sub = pd.DataFrame(sub)


100%|██████████| 253/253 [08:55<00:00,  2.11s/it]


In [17]:
metrics = calc_metrics(gt, sub)
metrics

{'Средний IoU рамок': 0.5858090129419741,
 'Средняя посимвольная точность текста': 0.8881395318693378,
 'Точность абсолютно верного распознавания текста': 0.12698412698412698}

In [49]:
score = calc_score(metrics)
score

0.622999117287954